# **ĐỌC DỮ LIỆU OPENSTREETMAP**

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("../data/data_DBScan.csv")
df

,datetime,vehicle,level_1,speed,y,x,heading,vehicleType,time_interval,distance,heading_interval,time,index_,CLUSTER_DBSCAN
0,2018-04-24 17:00:00,49A04047,0,10.0,106.692955,10.771090,61.161581,700,10.0,39.340379,141.522581,17:00:00,4000,0_0
1,2018-04-24 17:00:00,51F12268,0,24.0,106.690994,10.772771,119.289035,700,10.0,59.710443,47.134868,17:00:00,80854,0_1
2,2018-04-24 17:00:00,51F18403,0,15.0,106.689674,10.785602,221.276489,700,10.0,60.492194,113.858347,17:00:00,83202,0_2
3,2018-04-24 17:00:00,51F18428,0,15.0,106.697525,10.779303,44.207744,700,10.0,66.803232,84.559429,17:00:00,83244,0_3
4,2018-04-24 17:00:00,51F22973,0,21.0,106.691500,10.772524,298.144902,700,10.0,47.144122,145.417885,17:00:00,85390,0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154900,2018-04-24 23:59:38,51G00966,4,22.0,106.682495,10.779266,300.433548,300,12.0,72.967685,1.535565,23:59:38,157221,1258_36
154901,2018-04-24 23:59:38,51G39241,1,19.0,106.681620,10.765618,270.699827,300,4.0,45.227946,19.220497,23:59:38,176115,1258_38
154902,2018-04-24 23:59:39,50LD10215,0,17.0,106.676780,10.773272,239.686544,200,82258.0,1580.049776,1.776102,23:59:39,10270,1258_34
154903,2018-04-24 23:59:39,51G01373,4,19.0,106.689190,10.790087,306.825134,300,18.0,84.800954,1.510013,23:59:39,157825,1258_39


In [26]:
import osmium as osm
import pandas as pd

class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data_node = []
        self.osm_data_way = []
        self.osm_data_relation = []
        
    def tag_inventory(self, elem, elem_type):
        # lấy thông tin của node
        if elem_type == 'node':
            value = {'id':elem.id, 
                     'lat': elem.location.lat, 
                     'lon': elem.location.lon, 
                     'version': elem.version, 
                     'timestamp': pd.Timestamp(elem.timestamp), 
                     'changeset': elem.changeset,
                     'uid': elem.uid,
                     'user': elem.user}
            for tag in elem.tags:
                value[tag.k] = tag.v
            self.osm_data_node.append(value)
        
        # lấy thông tin của way
        elif elem_type == 'way':
            value = {'id':elem.id, 
                     'version': elem.version, 
                     'timestamp': pd.Timestamp(elem.timestamp), 
                     'changeset': elem.changeset,
                     'uid': elem.uid,
                     'user': elem.user}
            nds = []
            for nd in elem.nodes:
                nds.append(nd.ref)
            value['ref'] = nds
            
            for tag in elem.tags:
                value[tag.k] = tag.v
            self.osm_data_way.append(value)
        
        # lấy thông tin của relation
        else:
            for tag in elem.tags:
                 value = {'id':elem.id, 
                     'version': elem.version, 
                     'timestamp': pd.Timestamp(elem.timestamp), 
                     'changeset': elem.changeset,
                     'uid': elem.uid,
                     'user': elem.user}
            for tag in elem.tags:
                value[tag.k] = tag.v
            self.osm_data_relation.append(value)


    def node(self, n):
        self.tag_inventory(n, "node")

    def way(self, w):
        self.tag_inventory(w, "way")

    def relation(self, r):
        self.tag_inventory(r, "relation")


osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("../data/vnroads20201128_q3.osm")

In [35]:
# transform the list into a pandas DataFrame
df_osm_node = pd.DataFrame(osmhandler.osm_data_node)
#df_osm = tag_genome.sort_values(by=['type', 'id', 'timestamp'])
df_osm_node

,id,lat,lon,version,timestamp,changeset,uid,user,vbd:layer
0,1419383,10.788461,106.664307,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
1,1419404,10.789730,106.664316,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
2,1419414,10.787892,106.664322,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
3,1419415,10.766674,106.664324,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
4,1419423,10.792145,106.664329,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
...,...,...,...,...,...,...,...,...,...
20012,13599586,10.771815,106.697815,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
20013,13608426,10.777344,106.679361,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
20014,13608427,10.777360,106.679392,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad
20015,13608428,10.777361,106.679430,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad


In [36]:
df_osm_node.to_csv('../data/osm_node.csv', index = False)

In [29]:

df_osm_way = pd.DataFrame(osmhandler.osm_data_way)
#df_osm = tag_genome.sort_values(by=['type', 'id', 'timestamp'])
col_nan = df_osm_way.columns[df_osm_way.isna().any()].tolist()
df_osm_way = df_osm_way.drop(col_nan, axis = 1)
df_osm_way

,id,version,timestamp,changeset,uid,user,ref,vbd:layer,vbd:namebase,vbd:type,vbd:vtunnel,name,vbd:voverpass,vbd:level,highway
0,2560080,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1451209, 1450777]",vbdroad,Cao Thắng,3,0,Đường Cao Thắng,0,2,secondary
1,2560095,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1453773, 13416279, 13416280, 13416281, 1453627]",vbdroad,105 Đường Cao Thắng,5,0,Hẻm 105 Đường Cao Thắng,0,3,residential
2,2560109,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1453097, 13416288, 13416289, 13416290, 134162...",vbdroad,105 Đường Cao Thắng,5,0,Hẻm 105 Đường Cao Thắng,0,3,residential
3,2560113,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1452755, 13416307, 1453223]",vbdroad,129 Đường Cao Thắng,5,0,Hẻm 129 Đường Cao Thắng,0,3,residential
4,2560189,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1451462, 1452079]",vbdroad,Cư Xá Nguyễn Văn Trỗi,3,0,Đường Cư Xá Nguyễn Văn Trỗi,0,2,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10883,2714244,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1481237, 13599586, 1480757]",vbdroad,Phan Chu Trinh,3,0,Đường Phan Chu Trinh,0,2,secondary
10884,2714245,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1481237, 1482113]",vbdroad,Lê Lai,3,0,Đường Lê Lai,0,1,primary
10885,2719553,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1446056, 1446927]",vbdroad,Số 8,3,0,Đường Số 8,0,2,secondary
10886,2719554,1,2020-11-30 03:20:07+00:00,1,1,vietbando,"[1446927, 13608426, 13608427, 13608428, 136084...",vbdroad,Số 8,3,0,Đường Số 8,0,2,secondary


In [38]:
df_osm_way.to_csv('../data/osm_way.csv', index=False)

In [34]:
df_osm_relation = pd.DataFrame(osmhandler.osm_data_relation)
df_osm_relation

,id,version,timestamp,changeset,uid,user,vbd:layer,type,restriction,except
0,757,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_left_turn,motor;car;truck
1,758,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_left_turn,motor;car;truck
2,759,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_left_turn,motor;car;truck
3,761,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_left_turn,motor;car;truck
4,812,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_right_turn,motor;car;truck
...,...,...,...,...,...,...,...,...,...,...
78,2760,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_u_turn,motor
79,2761,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_u_turn,motor
80,2762,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_u_turn,motor
81,2763,1,2020-11-30 03:20:07+00:00,1,1,vietbando,vbdroad,restriction,no_u_turn,motor


In [39]:
df_osm_relation.to_csv('../data/osm_relation.csv', index= False)